In [1]:
'''
My Python and LSL project

Flow: 
    1.Change the screen from black to white-this way we can measure the screen onset time with a photocell
    2.Send an event over LSL  to mark the time of the screen change
    3.Record event markers and EEG data with the BrainProducts LSL interface.
'''

import time 
import datetime

# Importing the library
import pygame

 
# Initializing Pygame
pygame.init()



pygame 2.5.2 (SDL 2.28.3, Python 3.12.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


(5, 0)

In [23]:
import time
import pygame
from pygame.locals import *

pygame.init()

screen_width, screen_height = 1600, 900
screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption("white to black")


#variables for the colors
white = (255, 255, 255)
black = (0, 0, 0)

# Create a custom event to toggle between white and black
toggle_background_event = pygame.USEREVENT + 1 #custom event code number
pygame.time.set_timer(toggle_background_event, 1000)  # Generate the event every 1 second (it's in ms by default)

current_background = white  # sets the beginning to a white background

running = True 

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False   #makes sure event/program ends if window is closed
        elif event.type == toggle_background_event:   #when the event is running and not closed, it'll run the toggle event
            # Toggle between white and black backgrounds, defines the actual happenings in the event as changing background to either the defined white or black based on current background
            if current_background == white:
                current_background = black
            else:
                current_background = white

    screen.fill(current_background) # makes screen the current background, actually defines what current_background does
    pygame.display.flip()

pygame.quit() 




In [1]:
#can you just make this integrated and run good:

"""Example program to show how to read a multi-channel time series from LSL.
**has a while loop that just never ends if no data ?"""





import pandas as pd

            
import time
from pylsl import StreamInlet, resolve_stream

from pylsl import StreamInfo

eeg_mark = 0


from pylsl import StreamInfo

stream_name = "EEGStream"
stream_type = "EEG"
num_channels = 32
sampling_rate = 256

info = StreamInfo(stream_name, stream_type, num_channels, sampling_rate, 'float32', 'myuid34234')

from pylsl import StreamOutlet

outlet = StreamOutlet(info)

def main():
    # first resolve an EEG stream on the lab network
    print("looking for an EEG stream...")
    streams = resolve_stream('type', 'EEG')
    if not streams:
        print("No EEG streams found. Exiting.")
        return

#create a new inlet
    inlet = StreamInlet(streams[0])

    while True:   # this loop just doesn't end if it doesnt have any actual samples to pull, unsure if it will actually end with samples ?
        # get a new sample 
        sample, timestamp = inlet.pull_sample()
        if timestamp is None:
            print("No data received. Exiting.")
            def eeg_marker():
                eeg_mark +1

        break
        print(timestamp, sample) 
if __name__ == '__main__':
    main()


#input the screen changing code ? here it is as a comment

##screen turning white to black, set to 1000ms
import pygame
from pygame.locals import *

pygame.init()

screen_width, screen_height = 1600, 900
screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption("white to black")


#variables for the colors
white = (255, 255, 255)
black = (0, 0, 0)

# Create a custom event to toggle between white and black
color_change_event = pygame.USEREVENT + 2 #custom event code number
pygame.time.set_timer(toggle_background_event, timestamp + 1000)  # Generate the event based on time, should be dependent on the timestamp of when it's received ?

current_background = white  # sets the beginning to a white background

running = True 

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False   #makes sure event/program ends if window is closed
        elif event.type == color_change_event: 
            if main() == "start_task":
                current_background = black
            elif main() == "end_task":
                current_background = white

    screen.fill(current_background) # makes screen the current background, actually defines what current_background does
    pygame.display.flip()

pygame.quit() 



#Example program to demonstrate how to send a multi-channel time series to LSL

##send data example, doesn't really run properly because I assume because there's no EEG data currently on my PC

import sys
import getopt

import time
from random import random as rand

from pylsl import StreamInfo, StreamOutlet, local_clock


def main(argv):
    srate = 100
    name = 'BioSemi'
    type = 'EEG'
    n_channels = 8
    help_string = 'SendData.py -s <sampling_rate> -n <stream_name> -t <stream_type>'
    try:
        opts, args = getopt.getopt(argv, "hs:c:n:t:", longopts=["srate=", "channels=", "name=", "type"])
    except getopt.GetoptError:
        print(help_string)
        sys.exit(2)
    for opt, arg in opts:
        if opt == '-h':
            print(help_string)
            sys.exit()
        elif opt in ("-s", "--srate"):
            srate = float(arg)
        elif opt in ("-c", "--channels"):
            n_channels = int(arg)
        elif opt in ("-n", "--name"):
            name = arg
        elif opt in ("-t", "--type"):
            type = arg

    # first create a new stream info (here we set the name to BioSemi,
    # the content-type to EEG, 8 channels, 100 Hz, and float-valued data) The
    # last value would be the serial number of the device or some other more or
    # less locally unique identifier for the stream as far as available (you
    # could also omit it but interrupted connections wouldn't auto-recover)
    info = StreamInfo(name, type, n_channels, srate, 'float32', 'myuid34234')

    # next make an outlet
    outlet = StreamOutlet(info)

    print("now sending data...")
    start_time = local_clock()
    sent_samples = 0
    while True:
        elapsed_time = local_clock() - start_time
        required_samples = int(srate * elapsed_time) - sent_samples
        for sample_ix in range(required_samples):
            # make a new random n_channels sample; this is converted into a
            # pylsl.vectorf (the data type that is expected by push_sample)
            mysample = [rand() for _ in range(n_channels)]
            # now send it
            outlet.push_sample(mysample)
        sent_samples += required_samples
        # now send it and wait for a bit before trying again.
        time.sleep(0.01)

if __name__ == '__main__':
    main(sys.argv[1:])

looking for an EEG stream...


In [16]:
##functional LSL sending program for EEG data
## put in a .py file and run in terminal area


import sys
import getopt

import time
from random import random as rand

from pylsl import StreamInfo, StreamOutlet, local_clock

srate = 100
name = 'Python EEG Data'
type_stream = 'EEG'
n_channels = 32
help_string = 'SendData.py -s <sampling_rate> -n <stream_name> -t <stream_type>'

def main(argv):

    # first create a new stream info (here we set the name to BioSemi,
    # the content-type to EEG, 8 channels, 100 Hz, and float-valued data) The
    # last value would be the serial number of the device or some other more or
    # less locally unique identifier for the stream as far as available (you
    # could also omit it but interrupted connections wouldn't auto-recover)
    info = StreamInfo(name, type_stream, n_channels, srate, 'float32', 'myuid34234')

    # next make an outlet
    outlet = StreamOutlet(info)

    print("now sending data...")
    start_time = local_clock()
    sent_samples = 0
    while True:
        elapsed_time = local_clock() - start_time
        required_samples = int(srate * elapsed_time) - sent_samples
        for sample_ix in range(required_samples):
            # make a new random n_channels sample; this is converted into a
            # pylsl.vectorf (the data type that is expected by push_sample)
            mysample = [rand() for _ in range(n_channels)]
            # now send it
            outlet.push_sample(mysample)
        sent_samples += required_samples
        # now send it and wait for a bit before trying again.
        time.sleep(0.01)
    while False:
        print("EEG data sent!")
        break

if __name__ == '__main__':
    main(sys.argv[1:])


now sending data...


KeyboardInterrupt: 

In [21]:
#runs 
#the above code in a separate python file not in a jupyter notebook
# just a .py that still seems to clog the the LSL BrainVision viewer

!python SendingLSL.py

^C


In [1]:
# Record event markers and EEG data with the BrainProducts LSL interface.
    ## all on the BrainProducts LSL github


import sys
import os
import xml.etree.ElementTree as ET
import re
import ntpath

try:
	import configparser as CP
except ImportError:
	import ConfigParser as CP

bvefin_name = sys.argv[1]
configin_name = sys.argv[2]
num_args = len(sys.argv)
if num_args == 4:
	if sys.argv[3] == 'XML':
		is_conf_XML = True
	else:
		is_conf_XML = False
else:
	is_conf_XML = False

# setup output file name
con_name_bits = re.split('.cfg', configin_name)
bvefin_basename = ntpath.basename(bvefin_name)
bvef_name_bits = re.split('.bvef', bvefin_basename)
configout_name = con_name_bits[0] + '-' + bvef_name_bits[0] + '.cfg'

# parse bvef file
electrode_names = []
bv_tree = ET.parse(bvefin_name)
bv_root = bv_tree.getroot()
for electrode in bv_root.findall('Electrode'):
	# electrode = electrodes.find('Electrode')
	electrode_names.append(electrode.find('Name').text)

for n in electrode_names:
	if n == 'GND':
		electrode_names.remove('GND')
		break
for n in electrode_names:
	if n == 'REF':
		electrode_names.remove('REF')

for n in electrode_names:
	if n == 'Fpz Gnd':
		electrode_names.remove(n)
		break

for n in electrode_names:
	if n == 'FCz Ref':
		electrode_names.remove(n)
		break

# parse config file as xml and write new one
if is_conf_XML == True:

	# do this because LSL config files typically do not have roots
	# and python can't parse such xml files
	with open(configin_name) as f:
		xml = f.read()
	con_root = ET.fromstring(re.sub(r"(<\?xml[^>]+\?>)", r"\1<root>", xml) + "</root>")
	for channels in con_root.findall('channels'):
		con_root.remove(channels)

	channels = ET.SubElement(con_root, 'channels')
	labels = ET.SubElement(channels, 'labels')
	for electrode_name in electrode_names:
		ET.SubElement(labels, 'label').text = electrode_name

	s = ET.tostring(con_root).decode('utf-8')
	s = re.sub(r"<root>", "<?xml version=\"1.0\" encoding=\"utf-8\"?>", s)
	s = re.sub(r"</root>", "", s)
	f = open(configout_name, "w")
	f.write(s)

# parse config file as ini and write new one
else:

    cp_out = CP.ConfigParser()
    cp_in = CP.ConfigParser()
    cp_in.read(configin_name)
    str = ''
    for name in electrode_names[:-1]:
        str = str + name + ', '
    str = str + electrode_names[-1]
    try:
        for section in cp_in.sections():
            if section != 'channels':
                cp_out[section] = cp_in[section]

        cp_out['channels'] = {'labels': str}
    except AttributeError:
        for section in cp_in.sections():
            if section != 'channels':
                cp_out.add_section(section)
                for option in cp_in.options(section):
                    cp_out.set(section, option, cp_in.get(section, option))
        cp_out.add_section('channels')
        cp_out.set('channels', 'labels', str)
    with open(configout_name, 'w') as configfile:
        cp_out.write(configfile)


FileNotFoundError: [Errno 2] No such file or directory: '--ip=127.0.0.1'